In [1]:
!pip install keras
!pip install tensorflow
!pip install pydot

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import numpy as np
np.random.seed(13)

from keras.models import Sequential, Model
from keras.layers import Embedding, Reshape, Activation, Input
from keras.layers.merge import Dot
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import skipgrams

import gensim

Using TensorFlow backend.


In [3]:
path = get_file('alice.txt', origin='http://www.gutenberg.org/files/11/11-0.txt')
corpus = open(path).readlines()

corpus = [sentence for sentence in corpus if sentence.count(' ') >= 2]
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
V = len(tokenizer.word_index) + 1
V

3388

In [4]:
dim_embedddings = 128

# inputs
w_inputs = Input(shape=(1, ), dtype='int32')
w = Embedding(V, dim_embedddings)(w_inputs)

# context
c_inputs = Input(shape=(1, ), dtype='int32')
c  = Embedding(V, dim_embedddings)(c_inputs)
o = Dot(axes=2)([w, c])
o = Reshape((1,), input_shape=(1, 1))(o)
o = Activation('sigmoid')(o)

SkipGram = Model(inputs=[w_inputs, c_inputs], outputs=o)
SkipGram.summary()
SkipGram.compile(loss='binary_crossentropy', optimizer='adam')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 128)       433664      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 128)       433664      input_2[0][0]                    
__________________________________________________________________________________________________
dot_1 (Dot

In [5]:
for _ in range(5):
    loss = 0.
    for i, doc in enumerate(tokenizer.texts_to_sequences(corpus)):
        data, labels = skipgrams(sequence=doc, vocabulary_size=V, window_size=5, negative_samples=5.)
        x = [np.array(x) for x in zip(*data)]
        y = np.array(labels, dtype=np.int32)
        if i<4:
            print(' ', i, len(x), y.shape)
        if x:
            loss += SkipGram.train_on_batch(x, y)

    print(loss)

  0 2 (360,)
  1 2 (660,)
  2 2 (540,)
  3 2 (540,)
1078.7301915809512
  0 2 (360,)
  1 2 (660,)
  2 2 (540,)
  3 2 (540,)
743.1534576192498
  0 2 (360,)
  1 2 (660,)
  2 2 (540,)
  3 2 (540,)
689.5496688634157
  0 2 (360,)
  1 2 (660,)
  2 2 (540,)
  3 2 (540,)
662.8913199454546
  0 2 (360,)
  1 2 (660,)
  2 2 (540,)
  3 2 (540,)
635.4277692064643


In [6]:
f = open('vectors-sg-ns.txt' ,'w')
f.write('{} {}\n'.format(V-1, dim_embedddings))
vectors = SkipGram.get_weights()[0]
for word, i in tokenizer.word_index.items():
    f.write('{} {}\n'.format(word, ' '.join(map(str, list(vectors[i, :])))))
f.close()

In [7]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('./vectors-sg-ns.txt', binary=False)

In [8]:
w2v.most_similar(positive=['queen'])

[('king', 0.7530218362808228),
 ('hearts', 0.6878508925437927),
 ('cook', 0.6561591625213623),
 ('turning', 0.6399833559989929),
 ('knee', 0.6318691968917847),
 ('verse', 0.6307002902030945),
 ('wildly', 0.6285487413406372),
 ('mock', 0.6083570718765259),
 ('the', 0.608260989189148),
 ('trumpet', 0.590358316898346)]

In [9]:
w2v.most_similar(positive=['alice'])

[('thought', 0.5824406743049622),
 ('cautiously', 0.5794151425361633),
 ('‘i’m', 0.5747063159942627),
 ('indignantly', 0.573738694190979),
 ('‘but', 0.5686929821968079),
 ('rather', 0.5512331128120422),
 ('glad', 0.5424178242683411),
 ('curious', 0.5348520278930664),
 ('afraid', 0.5324457287788391),
 ('sharply', 0.5229484438896179)]

In [10]:
w2v.most_similar(positive=['the'])

[('hearts', 0.6303597092628479),
 ('queen', 0.608260989189148),
 ('king', 0.5960526466369629),
 ('verse', 0.5643625259399414),
 ('queen’s', 0.5551983714103699),
 ('cook', 0.5540987253189087),
 ('place', 0.5464391112327576),
 ('laws', 0.5327359437942505),
 ('s', 0.5246920585632324),
 ('list', 0.521309494972229)]